In [76]:
from opensearchpy import OpenSearch
import re

In [3]:
# OpenSearch 연결 설정
client = OpenSearch(
    hosts=[{"host": "localhost", "port": 9200}],  # OpenSearch 주소
    http_auth=("admin", "admin"),                 # 보안 계정 (있다면)
    use_ssl=False,
    verify_certs=False
)

## 재무정보 잘 들어갔나 확인

In [125]:
result = client.search(
    index="rpt_sec_eq",
    body={
        "_source": ["corp_code", "induty_code", "financials"],  # induty_code 추가
        "query": {
            "exists": {"field": "financials"}
        }
    }
)

for hit in result["hits"]["hits"]:
    source = hit["_source"]
    print(
        f"corp_code: {source.get('corp_code')}, "
        f"induty_code: {source.get('induty_code')}, "
        f"financials: {source.get('financials')}"
    )


corp_code: 00113207, induty_code: 28302, financials: {'cash_flow_financing': 411055519000.0, 'cash_flow_operating': 6751598000.0, 'total_assets': 2642726208000.0, 'net_margin': -0.021403535799595867, 'operating_income': -115169757000.0, 'revenue_growth': 0.15728639481028814, 'equity_ratio': 0.5661246584950809, 'operating_margin': -0.034992185115339136, 'debt_to_equity': 0.7663954130849595, 'equity': 1496112472000.0, 'revenue': 3291299375000.0, 'cash_flow_investing': -387108449000.0, 'operating_income_growth': 0.4427981383217014, 'net_income': -70445444000.0, 'operating_cf_to_investing_cf': -0.017441102144479415, 'operating_cf_to_revenue': 0.002051347273749596}
corp_code: 00260657, induty_code: 271, financials: {'cash_flow_financing': -7472794085.0, 'cash_flow_operating': -38767362947.0, 'total_assets': 295000202043.0, 'net_margin': -0.14119361737454442, 'operating_income': -24784976694.0, 'revenue_growth': -0.773811803481803, 'equity_ratio': 0.9695518149486192, 'operating_margin': -0.3

In [121]:
result = client.search(
    index="rpt_sec_eq",
    body={
        "_source": ["corp_code", "financials"],
        "query": {
            "exists": {"field": "financials"}
        }
    }
)

for hit in result["hits"]["hits"]:
    print(hit["_source"])

## 1차로 절대적 수치를 통한 필터링, 2차로 비율로 필터링

In [81]:
# 1. Dummy 데이터
dummy_float = {
    "revenue": 125000000000.0,
    "operating_income": 18500000000.0,
    "net_income": 12300000000.0,
    "total_assets": 310000000000.0,
    "equity": 210000000000.0,
    "cash_flow_operating": 22000000000.0,
    "cash_flow_investing": -15500000000.0,
    "cash_flow_financing": -7800000000.0,
    "operating_margin": 0.148,
    "net_margin": 0.0984,
    "debt_to_equity": 0.476,
    "equity_ratio": 0.677,
    "operating_cf_to_investing_cf": -1.419,
    "operating_cf_to_revenue": 0.176,
    "revenue_growth": 0.052,
    "operating_income_growth": 0.163    
}

# 1차 필터 기준 허용 오차 (% 단위 예시)
ABS_TOLERANCE = 0.9  # ±50% 범위 내 필터

query = {
    "size": 5,
    "query": {
        "script_score": {
            "query": {
                "bool": {
                    "filter": [
                        {"range": {"financials.revenue": {"gte": dummy_float["revenue"] * (1 - ABS_TOLERANCE),
                                                          "lte": dummy_float["revenue"] * (1 + ABS_TOLERANCE)}}},
                        {"range": {"financials.total_assets": {"gte": dummy_float["total_assets"] * (1 - ABS_TOLERANCE),
                                                               "lte": dummy_float["total_assets"] * (1 + ABS_TOLERANCE)}}},
                        {"range": {"financials.equity": {"gte": dummy_float["equity"] * (1 - ABS_TOLERANCE),
                                                         "lte": dummy_float["equity"] * (1 + ABS_TOLERANCE)}}}
                    ]
                }
            },
            "script": {
                "source": """
                    double score = 0;
                    score += 1 / (1 + Math.abs(doc['financials.operating_margin'].value - params.operating_margin));
                    score += 1 / (1 + Math.abs(doc['financials.net_margin'].value - params.net_margin));
                    score += 1 / (1 + Math.abs(doc['financials.debt_to_equity'].value - params.debt_to_equity));
                    score += 1 / (1 + Math.abs(doc['financials.equity_ratio'].value - params.equity_ratio));
                    score += 1 / (1 + Math.abs(doc['financials.operating_cf_to_investing_cf'].value - params.operating_cf_to_investing_cf));
                    score += 1 / (1 + Math.abs(doc['financials.operating_cf_to_revenue'].value - params.operating_cf_to_revenue));
                    score += 1 / (1 + Math.abs(doc['financials.revenue_growth'].value - params.revenue_growth));
                    score += 1 / (1 + Math.abs(doc['financials.operating_income_growth'].value - params.operating_income_growth));
                    return score;
                """,
                "params": dummy_float
            }
        }
    },
    "collapse": {
        "field": "corp_code"   # 중복 제거 기준
    }
}



# 4. 실행
response = client.search(index="rpt_sec_eq", body=query)

# 5. 결과 출력
for hit in response["hits"]["hits"]:
    print(hit["_source"]["corp_code"], hit["_source"]["financials"])


corp_codes = [hit["_source"]["corp_code"] for hit in response["hits"]["hits"]]


00328191 {'cash_flow_financing': 14615349731.0, 'cash_flow_operating': 17544632590.0, 'total_assets': 105512995900.0, 'net_margin': -0.17933180034546573, 'operating_income': -13810410136.0, 'revenue_growth': 0.04457388849908761, 'equity_ratio': 0.676588788888706, 'operating_margin': -0.16197029533413007, 'debt_to_equity': 0.47800261609787453, 'equity': 71388910108.0, 'revenue': 85265079671.0, 'cash_flow_investing': -17305142545.0, 'operating_income_growth': 0.16658971362123215, 'net_income': -15290740244.0, 'operating_cf_to_investing_cf': -1.0138392413918136, 'operating_cf_to_revenue': 0.20576574440200995}
01030503 {'cash_flow_financing': 8016674738.0, 'cash_flow_operating': 7984282891.0, 'total_assets': 51748720293.0, 'net_margin': -0.12305278328774147, 'operating_income': -5318973279.0, 'revenue_growth': 0.08052167029382583, 'equity_ratio': 0.916420937802687, 'operating_margin': -0.12099973558658714, 'debt_to_equity': 0.09120160697955185, 'equity': 47423610781.0, 'revenue': 439585529

In [85]:
def get_sec_content_re(title, corp_code):
    # 쿼리 정의
    query = {
        "_source": False,     # 문서 전체 가져오지 않음
        "from": 0,
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {   # corp_code 필터
                        "term": {
                            "corp_code": corp_code
                        }
                    },
                    {   # nested sections 검색
                        "nested": {
                            "path": "sections",
                            "query": {
                                "bool": {
                                    "should": [
                                        {
                                            "match_phrase": {
                                                "sections.sec_title": {
                                                    "query": f"{title}",
                                                    "boost": 1.0
                                                }
                                            }
                                        }
                                    ]
                                }
                            },
                            "inner_hits": {
                                "_source": ["sections.sec_id", "sections.sec_title", "sections.sec_content"]
                            }
                        }
                    }
                ]
            }
        }
    }

    response = client.search(
        body=query,
        index="rpt_sec_eq"
    )

    for hit in response["hits"]["hits"]:
        inner_hits = hit["inner_hits"]["sections"]["hits"]["hits"]
        for section in inner_hits:
            src = section["_source"]
            content = src['sec_content']

            cleaned_content = re.sub(r"</?p>", "", content)

            # <td> ~ </td> 블록 추출
            td_blocks = re.findall(r"<td>(.*?)</td>|<td><p>(.*?)</p></td>", cleaned_content, flags=re.DOTALL)
            td_texts = [block[0] if block[0] else block[1] for block in td_blocks]

            for text in td_texts:
                # 항목 번호가 맨 앞에 있는 경우만 매칭
                items = re.findall(
                    r"^(가\.|나\.|다\.|라\.|마\.|바\.|사\.|아\.|자\.|차\.|카\.|타\.|파\.|하\.)\s*(.*?)(?=(?:\n(가\.|나\.|다\.|라\.|마\.|바\.|사\.|아\.|자\.|차\.|카\.|타\.|파\.|하\.)|$))",
                    text,
                    flags=re.DOTALL | re.MULTILINE
                )

                for item in items:
                    print(f"{item[0]} {item[1].strip()}")
            print("-"*50)
    return response


In [ ]:
corp_codes

['00328191', '01030503', '00126414', '00309460', '00260657']

In [87]:
for i in range(len(corp_codes)):
    get_sec_content_re("회사위험", corp_codes[i])

가. 성장성 관련 위험당사는 조선업의 업황 개선에 따라, 최근 3년간 매출액이 개선되고 있는 추세이며, 수주잔고 또한 지속적으로 증가하고 있습니다. 이에, 생산시설의 가동시간을 늘려 증가하는 주문에 대응하고 있습니다. 이의 결과로, 당사의 재고자산의 수준이 증가하여 총자산의 규모도 확대되고 있습니다. 현재 전사 차원의 가동률은 89.67%로 매우 높은 수준을 유지하고 있습니다. 다만, 별도의 증설이 없을 경우, 당사는 추가 주문에 대응하지 못하거나 납기를 맞추지 못하여 매출 성장에 부정적인 영향을 받을 수 있습니다. 또한, 최근 3년간 당사의 성장성은 전방산업인 조선업의 업황 개선에 따라 유지될 수 있었기에, 향후 조선업의 업황이 악화된다면, 이는 당사의 성장성에 부정적인 영향을 줄 수 있습니다. 투자자께서는 이 점 유의하시기 바랍니다.
나. 수익성 관련 위험당사의 최근 3년 매출액, 영업이익, 당기순이익은 성장하고 있는 추세입니다. 이는 전방산업의 경기 호조에 따른 물량 증가, 원재료 상승분을 판가에 반영함에 따른 수익성 방어 등에 기인합니다. 당사의 주요 사업은 원재료를 구입하여 이를 보유하고 있는 생산 설비를 통해 가공하는 사업으로 비용 중 원재료 비의 비중이 매우 높은 사업 구조를 가지고 있습니다. 이로 인해 당사의 매출원가율은 약 90% 수준을 유지하고 있습니다. 높은 매출원가율에도, 당사는 약 5% 수준의 판관비율을 유지하고 있으며, 이를 통해 약 5% 수준의 영업이익율을 유지하고 있습니다. 환율의 변동 등에 따른 기타손익의 변동 및 단기차입금에서 발생하는 이자비용 등의 영향에도 최근 3년 간 당기순이익이 발생하였습니다. 다만, 과거 주요 원재료의 가격이 니켈 가격 급등으로 인해 크게 증가한 이력이 존재합니다. 이 때 판매단가 협상으로 원재료 가격 상승분을 일부 판매단가에 반영함에 따라, 수익성을 확보할 수 있었습니다. 그러나, 조선 업종의 경기가 악화되어 원재료 가격 상승분을 판매단가에 반영하지 못할 경우, 이는 당사의 수익성에 부정적인 영향을 줄

In [72]:
response = get_sec_content_re("회사위험", "00328191")

가. 성장성 관련 위험당사는 조선업의 업황 개선에 따라, 최근 3년간 매출액이 개선되고 있는 추세이며, 수주잔고 또한 지속적으로 증가하고 있습니다. 이에, 생산시설의 가동시간을 늘려 증가하는 주문에 대응하고 있습니다. 이의 결과로, 당사의 재고자산의 수준이 증가하여 총자산의 규모도 확대되고 있습니다. 현재 전사 차원의 가동률은 89.67%로 매우 높은 수준을 유지하고 있습니다. 다만, 별도의 증설이 없을 경우, 당사는 추가 주문에 대응하지 못하거나 납기를 맞추지 못하여 매출 성장에 부정적인 영향을 받을 수 있습니다. 또한, 최근 3년간 당사의 성장성은 전방산업인 조선업의 업황 개선에 따라 유지될 수 있었기에, 향후 조선업의 업황이 악화된다면, 이는 당사의 성장성에 부정적인 영향을 줄 수 있습니다. 투자자께서는 이 점 유의하시기 바랍니다.
나. 수익성 관련 위험당사의 최근 3년 매출액, 영업이익, 당기순이익은 성장하고 있는 추세입니다. 이는 전방산업의 경기 호조에 따른 물량 증가, 원재료 상승분을 판가에 반영함에 따른 수익성 방어 등에 기인합니다. 당사의 주요 사업은 원재료를 구입하여 이를 보유하고 있는 생산 설비를 통해 가공하는 사업으로 비용 중 원재료 비의 비중이 매우 높은 사업 구조를 가지고 있습니다. 이로 인해 당사의 매출원가율은 약 90% 수준을 유지하고 있습니다. 높은 매출원가율에도, 당사는 약 5% 수준의 판관비율을 유지하고 있으며, 이를 통해 약 5% 수준의 영업이익율을 유지하고 있습니다. 환율의 변동 등에 따른 기타손익의 변동 및 단기차입금에서 발생하는 이자비용 등의 영향에도 최근 3년 간 당기순이익이 발생하였습니다. 다만, 과거 주요 원재료의 가격이 니켈 가격 급등으로 인해 크게 증가한 이력이 존재합니다. 이 때 판매단가 협상으로 원재료 가격 상승분을 일부 판매단가에 반영함에 따라, 수익성을 확보할 수 있었습니다. 그러나, 조선 업종의 경기가 악화되어 원재료 가격 상승분을 판매단가에 반영하지 못할 경우, 이는 당사의 수익성에 부정적인 영향을 줄